<a href="https://colab.research.google.com/github/robgon-art/DeepHaiku-LLaMa/blob/main/2_Deep_Haiku_Quantize_Model_to_GGML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

In [2]:
MODEL_ID = "robgonsalves/llama-2-13b-deep-haiku"

# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'llama-2-13b-deep-haiku'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (25/25), 481.28 KiB | 2.24 MiB/s, done.
Filtering content: 100% (3/3), 4.24 GiB | 3.96 MiB/s, done.
Encountered 3 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00003-of-00003.bin
	pytorch_model-00001-of-00003.bin
	pytorch_model-00002-of-00003.bin

See: `git lfs help smudge` for more details.


In [4]:
MODEL_NAME = MODEL_ID.split('/')[-1]
GGML_VERSION = "ggml"

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16}

Loading model file llama-2-13b-deep-haiku/pytorch_model-00001-of-00003.bin
Loading model file llama-2-13b-deep-haiku/pytorch_model-00001-of-00003.bin
Loading model file llama-2-13b-deep-haiku/pytorch_model-00002-of-00003.bin
Loading model file llama-2-13b-deep-haiku/pytorch_model-00003-of-00003.bin
params = Params(n_vocab=32000, n_embd=5120, n_layer=40, n_ctx=4096, n_ff=13824, n_head=40, n_head_kv=40, f_norm_eps=1e-05, f_rope_freq_base=None, f_rope_scale=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('llama-2-13b-deep-haiku'))
Loading vocab file 'llama-2-13b-deep-haiku/tokenizer.model', type 'spm'
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting 

In [5]:
QUANTIZATION_METHODS = ["q5_k_m"]

for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.{method}.bin"
    !./llama.cpp/quantize {fp16} {qtype} {method}

ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
main: build = 1270 (c091cdf)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'llama-2-13b-deep-haiku/llama-2-13b-deep-haiku.ggml.fp16.bin' to 'llama-2-13b-deep-haiku/llama-2-13b-deep-haiku.ggml.q5_k_m.bin' as Q5_K_M
llama_model_loader: loaded meta data with 18 key-value pairs and 363 tensors from llama-2-13b-deep-haiku/llama-2-13b-deep-haiku.ggml.fp16.bin (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight f16      [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight f16      [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight f16      [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.a

In [6]:
!pip install -q huggingface_hub

username = "your-hugging-face-username"

from huggingface_hub import notebook_login, create_repo, HfApi
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.8 MB/s eta 0:00:00


In [7]:
api = HfApi()

# Create repo
create_repo(
    repo_id=f"{username}/{MODEL_NAME}-GGML",
    repo_type="model",
    exist_ok=True
)

# Upload bin models
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGML",
    allow_patterns=f"*{GGML_VERSION}*",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

llama-2-13b-deep-haiku.ggml.fp16.bin:   0%|          | 0.00/26.0G [00:00<?, ?B/s]

llama-2-13b-deep-haiku.ggml.q5_k_m.bin:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

'https://huggingface.co/robgonsalves/llama-2-13b-deep-haiku-GGML/tree/main/'